In [2]:
import pandas as pd

In [3]:
maos = pd.read_csv("../data/scotland/maos.csv")
dz_lookup = pd.read_csv('../data/scotland/2011_dz_lookup.csv')
pc_lookup = pd.read_csv("../data/scotland/postcode_lookup.csv", low_memory=False)
pc_lookup.drop_duplicates(subset="Postcode", inplace=True)
pop_est_2020 = pd.read_csv("../data/scotland/mid-year-pop-est-2020.csv")

maos_postcode = pd.merge(maos, pc_lookup, left_on="postcode", right_on="Postcode")
maos_postcode_la = pd.merge(maos_postcode, dz_lookup, left_on='DataZone2011Code', right_on='DZ2011_Code', how="left")
maos_postcode_la = maos_postcode_la.loc[:, ["name", "location_coord.lng", "location_coord.lat", "postcode", "LA_Name"]].copy()
maos_by_la = maos_postcode_la.groupby("LA_Name")["name"].count().reset_index()
maos_by_la.rename(columns={"name": "Number of MAOs"}, inplace=True)

pop_est_2020 = pop_est_2020.loc[:, ["Area name", "All Ages"]].copy()
pop_mao = pop_est_2020.merge(maos_by_la, left_on="Area name", right_on="LA_Name", how="left")
pop_mao.drop(columns=["LA_Name"], inplace=True)
pop_mao.fillna(0.0, inplace=True)
pop_mao["All Ages"] = pop_mao["All Ages"].apply(lambda x: int(x.replace(",", "")))
pop_mao["MAOs per 100k people"] = ((pop_mao["Number of MAOs"] / pop_mao["All Ages"]) * 100000).round(1)

pop_mao.at[19, "Area name"] = "Na h-Eileanan Siar"
pop_mao

,Area name,All Ages,Number of MAOs,MAOs per 100k people
0,Aberdeen City,229060,1.0,0.4
1,Aberdeenshire,260780,18.0,6.9
2,Angus,115820,5.0,4.3
3,Argyll and Bute,85430,4.0,4.7
4,City of Edinburgh,527620,33.0,6.3
5,Clackmannanshire,51290,1.0,1.9
6,Dumfries and Galloway,148290,21.0,14.2
7,Dundee City,148820,2.0,1.3
8,East Ayrshire,121600,6.0,4.9
9,East Dunbartonshire,108750,3.0,2.8


In [7]:
shs = pd.read_csv("../data/scotland/SHS Data Explorer.csv")
shs = shs.loc[:, ["Council", "All"]].copy()
shs.rename(columns={"All": "I could rely on someone in my neighbourhood to help me (%)"}, inplace=True)
shs.drop(shs[shs["Council"] == "Scotland"].index, inplace=True)

shs["Council"].at[11] = "City of Edinburgh"
shs.at[19, "Council"] = "Na h-Eileanan Siar"
shs = shs.sort_values("Council").reset_index(drop=True)

pop_mao_shs = pd.merge(pop_mao, shs, left_on="Area name", right_on="Council", how="right")
pop_mao_shs.drop(columns="Council", inplace=True)
pop_mao_shs.to_csv("../data/scotland/shs_mao.csv")

In [8]:
pop_mao_shs = pd.read_csv("../data/scotland/shs_mao.csv")

pop_mao_aberdeen_city = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Aberdeen City"].copy()
pop_mao_aberdeen_city.to_csv("../data/aberdeen_city/mao_shs_aberdeen_city.csv", index=False)

pop_mao_aberdeenshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Aberdeenshire"].copy()
pop_mao_aberdeenshire.to_csv("../data/aberdeenshire/mao_shs_aberdeenshire.csv", index=False)

pop_mao_angus = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Angus"].copy()
pop_mao_angus.to_csv("../data/angus/mao_shs_angus.csv", index=False)

pop_mao_argyll_and_bute = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Argyll and Bute"].copy()
pop_mao_argyll_and_bute.to_csv("../data/argyll_and_bute/mao_shs_argyll_and_bute.csv", index=False)

pop_mao_city_of_edinburgh = pop_mao_shs.loc[pop_mao_shs["Area name"] == "City of Edinburgh"].copy()
pop_mao_city_of_edinburgh.to_csv("../data/city_of_edinburgh/mao_shs_city_of_edinburgh.csv", index=False)

pop_mao_clackmannanshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Clackmannanshire"].copy()
pop_mao_clackmannanshire.to_csv("../data/clackmannanshire/mao_shs_clackmannanshire.csv", index=False)

pop_mao_dumfries_and_galloway = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Dumfries and Galloway"].copy()
pop_mao_dumfries_and_galloway.to_csv("../data/dumfries_and_galloway/mao_shs_dumfries_and_galloway.csv", index=False)

pop_mao_dundee_city = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Dundee City"].copy()
pop_mao_dundee_city.to_csv("../data/dundee_city/mao_shs_dundee_city.csv", index=False)

pop_mao_east_ayrshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "East Ayrshire"].copy()
pop_mao_east_ayrshire.to_csv("../data/east_ayrshire/mao_shs_east_ayrshire.csv", index=False)

pop_mao_east_dunbartonshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "East Dunbartonshire"].copy()
pop_mao_east_dunbartonshire.to_csv("../data/east_dunbartonshire/mao_shs_east_dunbartonshire.csv", index=False)

pop_mao_east_lothian = pop_mao_shs.loc[pop_mao_shs["Area name"] == "East Lothian"].copy()
pop_mao_east_lothian.to_csv("../data/east_lothian/mao_shs_east_lothian.csv", index=False)

pop_mao_east_renfrewshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "East Lothian"].copy()
pop_mao_east_renfrewshire.to_csv("../data/east_renfrewshire/mao_shs_east_renfrewshire.csv", index=False)

pop_mao_falkirk = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Falkirk"].copy()
pop_mao_falkirk.to_csv("../data/falkirk/mao_shs_falkirk.csv", index=False)

pop_mao_fife = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Fife"].copy()
pop_mao_fife.to_csv("../data/fife/mao_shs_fife.csv", index=False)

pop_mao_fife = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Fife"].copy()
pop_mao_fife.to_csv("../data/fife/mao_shs_fife.csv", index=False)

pop_mao_glasgow_city = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Glasgow City"].copy()
pop_mao_glasgow_city.to_csv("../data/glasgow_city/mao_shs_glasgow_city.csv", index=False)

pop_mao_highland = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Highland"].copy()
pop_mao_highland.to_csv("../data/highland/mao_shs_highland.csv", index=False)

pop_mao_inverclyde = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Inverclyde"].copy()
pop_mao_inverclyde.to_csv("../data/inverclyde/mao_shs_inverclyde.csv", index=False)

pop_mao_midlothian = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Midlothian"].copy()
pop_mao_midlothian.to_csv("../data/midlothian/mao_shs_midlothian.csv", index=False)

pop_mao_moray = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Moray"].copy()
pop_mao_moray.to_csv("../data/moray/mao_shs_moray.csv", index=False)

pop_mao_na_h_eileanan_siar = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Na h-Eileanan Siar"].copy()
pop_mao_na_h_eileanan_siar.to_csv("../data/na_h-eileanan_siar/mao_shs_na_h-eileanan_siar.csv", index=False)

pop_mao_north_ayrshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "North Ayrshire"].copy()
pop_mao_north_ayrshire.to_csv("../data/north_ayrshire/mao_shs_north_ayrshire.csv", index=False)

pop_mao_north_lanarkshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "North Lanarkshire"].copy()
pop_mao_north_lanarkshire.to_csv("../data/north_lanarkshire/mao_shs_north_lanarkshire.csv", index=False)

pop_mao_orkney_islands = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Orkney Islands"].copy()
pop_mao_orkney_islands.to_csv("../data/orkney_islands/mao_shs_orkney_islands.csv", index=False)

pop_mao_perth_and_kinross = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Perth and Kinross"].copy()
pop_mao_perth_and_kinross.to_csv("../data/perth_and_kinross/mao_shs_perth_and_kinross.csv", index=False)

pop_mao_renfrewshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Renfrewshire"].copy()
pop_mao_renfrewshire.to_csv("../data/renfrewshire/mao_shs_renfrewshire.csv", index=False)

pop_mao_scottish_borders = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Scottish Borders"].copy()
pop_mao_scottish_borders.to_csv("../data/scottish_borders/mao_shs_scottish_borders.csv", index=False)

pop_mao_shetland_islands = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Shetland Islands"].copy()
pop_mao_shetland_islands.to_csv("../data/shetland_islands/mao_shs_shetland_islands.csv", index=False)

pop_mao_south_ayrshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "South Ayrshire"].copy()
pop_mao_south_ayrshire.to_csv("../data/south_ayrshire/mao_shs_south_ayrshire.csv", index=False)
 
pop_mao_south_lanarkshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "South Lanarkshire"].copy()
pop_mao_south_lanarkshire.to_csv("../data/south_lanarkshire/mao_shs_south_lanarkshire.csv", index=False)

pop_mao_stirling = pop_mao_shs.loc[pop_mao_shs["Area name"] == "Stirling"].copy()
pop_mao_stirling.to_csv("../data/stirling/mao_shs_stirling.csv", index=False)

pop_mao_west_dunbartonshire = pop_mao_shs.loc[pop_mao_shs["Area name"] == "West Dunbartonshire"].copy()
pop_mao_west_dunbartonshire.to_csv("../data/west_dunbartonshire/mao_shs_west_dunbartonshire.csv", index=False)

pop_mao_west_lothian = pop_mao_shs.loc[pop_mao_shs["Area name"] == "West Lothian"].copy()
pop_mao_west_lothian.to_csv("../data/west_lothian/mao_shs_west_lothian.csv", index=False)